In [85]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pylab as plt
import seaborn as sns
import warnings
warnings.simplefilter("ignore")
import scipy as sp
from scipy import stats

from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
from xgboost import XGBClassifier
import xgboost as xgb
import datetime
from sklearn.preprocessing import LabelEncoder

import os
import gc
import random

%matplotlib inline

pd.options.display.max_rows = 10000
pd.options.display.max_columns = 10000
pd.options.display.max_colwidth = 1000

In [2]:
%%time
path = "I:/ML Datasets/IEEE Fradulent/train_identity.csv"
train_identity = pd.read_csv(path)

path = "I:/ML Datasets/IEEE Fradulent/train_transaction.csv"
train_transaction = pd.read_csv(path)

path = "I:/ML Datasets/IEEE Fradulent/test_identity.csv"
test_identity= pd.read_csv(path)

path = "I:/ML Datasets/IEEE Fradulent/test_transaction.csv"
test_transaction = pd.read_csv(path)

train = train_transaction.merge(train_identity, how='left', on='TransactionID')
test = test_transaction.merge(test_identity, how='left', on='TransactionID')

#path = "/content/drive/My Drive/Dataset/IEEE Fradulent/sample_submission.csv"
#df_submission = pd.read_csv(path)

Wall time: 1min 24s


In [3]:
ccols = [f'C{i}' for i in range(1,15)] ## Creating columns C1,C2....C14
dcols = [f'D{i}' for i in range(1,16)] ## Creating columns D1,D2....D14

cols = [
    'TransactionID','isFraud','TransactionDT','TransactionAmt','ProductCD',
    'card1','card2','card3','card4','card5','card6',
    'addr1','addr2','dist1','dist2',
    'P_emaildomain','R_emaildomain',
    
]

cols += dcols
cols += ccols

In [5]:
test[ccols].isna().sum()  ## Finding all those C columns that have Na values and replacing them with 0's//
#Note: the train columns (C1-C14) have no Nan values and therefore they dont have to be changed

C1        3
C2        3
C3        3
C4        3
C5        3
C6        3
C7        3
C8        3
C9        3
C10       3
C11       3
C12       3
C13    4748
C14       3
dtype: int64

In [9]:
test[ccols] = test[ccols].fillna(0)
test[ccols].isna().sum()

C1     0
C2     0
C3     0
C4     0
C5     0
C6     0
C7     0
C8     0
C9     0
C10    0
C11    0
C12    0
C13    0
C14    0
dtype: int64

In [10]:
## Had to rename these columns because there were differnt in the test and the train columns

test =  test.rename(columns = {'id-01':'id_01', 'id-02':'id_02', 'id-03':'id_03', 'id-04':'id_04', 'id-05':'id_05', 'id-06':'id_06','id-07':'id_07','id-08':'id_08',
     'id-09':'id_09','id-10':'id_10','id-11':'id_11', 'id-12':'id_12', 'id-13':'id_13', 'id-14':'id_14', 'id-15':'id_15', 'id-16':'id_16',
       'id-17':'id_17', 'id-18':'id_18','id-19':'id_19', 'id-20':'id_20', 'id-21':'id_21','id-22':'id_22','id-23':'id_23', 'id-24':'id_24',
      'id-25':'id_25', 'id-26':'id_26', 'id-27':'id_27', 'id-28':'id_28', 'id-29':'id_29','id-30':'id_30','id-31':'id_31','id-32':'id_32',
      'id-33':'id_33','id-34':'id_34', 'id-35':'id_35','id-36':'id_36','id-37':'id_37','id-38':'id_38'})

In [11]:
test['isFraud'] = 0
#test = test[train.columns]

train_test = pd.concat([train, test], axis=0)
print(train_test.shape)

del train, test
gc.collect()

(1097231, 434)


105

As to the reason why we are using this function to reduce memory usage there is a briliiant article on this in the website
https://www.kaggle.com/alexeykupershtokh/safe-memory-reduction

In [12]:
# safe downcast
def sd(col, max_loss_limit=0.001, avg_loss_limit=0.001, na_loss_limit=0, n_uniq_loss_limit=0, fillna=0):
    """
    max_loss_limit - don't allow any float to lose precision more than this value. Any values are ok for GBT algorithms as long as you don't unique values.
                     See https://en.wikipedia.org/wiki/Half-precision_floating-point_format#Precision_limitations_on_decimal_values_in_[0,_1]
    avg_loss_limit - same but calculates avg throughout the series.
    na_loss_limit - not really useful.
    n_uniq_loss_limit - very important parameter. If you have a float field with very high cardinality(high number of unique
    values) you can set this value to something like n_records * 0.01 in order to allow some field relaxing.
    """
    is_float = str(col.dtypes)[:5] == 'float'
    na_count = col.isna().sum()
    n_uniq = col.nunique(dropna=False)
    try_types = ['float16', 'float32']

    if na_count <= na_loss_limit:
        try_types = ['int8', 'int16', 'float16', 'int32', 'float32']

    for type in try_types:
        col_tmp = col

        # float to int conversion => try to round to minimize casting error
        if is_float and (str(type)[:3] == 'int'):
            col_tmp = col_tmp.copy().fillna(fillna).round()

        col_tmp = col_tmp.astype(type)
        max_loss = (col_tmp - col).abs().max()
        avg_loss = (col_tmp - col).abs().mean()
        na_loss = np.abs(na_count - col_tmp.isna().sum())
        n_uniq_loss = np.abs(n_uniq - col_tmp.nunique(dropna=False))

        if max_loss <= max_loss_limit and avg_loss <= avg_loss_limit and na_loss <= na_loss_limit and n_uniq_loss <= n_uniq_loss_limit:
            return col_tmp

    # field can't be converted
    return col

In [14]:
def reduce_mem_usage_sd(df, deep=True, verbose=False, obj_to_cat=False):
    numerics = ['int16', 'uint16', 'int32', 'uint32', 'int64', 'uint64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=deep).sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes

        # collect stats
        na_count = df[col].isna().sum()
        n_uniq = df[col].nunique(dropna=False)
        
        # numerics
        if col_type in numerics:
            df[col] = sd(df[col])

        # strings
        if (col_type == 'object') and obj_to_cat:
            df[col] = df[col].astype('category')
        
        if verbose:
            print(f'Column {col}: {col_type} -> {df[col].dtypes}, na_count={na_count}, n_uniq={n_uniq}')
        new_na_count = df[col].isna().sum()
        if (na_count != new_na_count):
            print(f'Warning: column {col}, {col_type} -> {df[col].dtypes} lost na values. Before: {na_count}, after: {new_na_count}')
        new_n_uniq = df[col].nunique(dropna=False)
        if (n_uniq != new_n_uniq):
            print(f'Warning: column {col}, {col_type} -> {df[col].dtypes} lost unique values. Before: {n_uniq}, after: {new_n_uniq}')

    end_mem = df.memory_usage(deep=deep).sum() / 1024 ** 2
    percent = 100 * (start_mem - end_mem) / start_mem
    if verbose:
        print('Mem. usage decreased from {:5.2f} Mb to {:5.2f} Mb ({:.1f}% reduction)'.format(start_mem, end_mem, percent))
    return df

In [15]:
train_test = reduce_mem_usage_sd(train_test, verbose=True)

Column C1: float64 -> int16, na_count=0, n_uniq=2187
Column C10: float64 -> int16, na_count=0, n_uniq=1387
Column C11: float64 -> int16, na_count=0, n_uniq=1834
Column C12: float64 -> int16, na_count=0, n_uniq=1530
Column C13: float64 -> int16, na_count=0, n_uniq=1691
Column C14: float64 -> int16, na_count=0, n_uniq=1150
Column C2: float64 -> int16, na_count=0, n_uniq=1784
Column C3: float64 -> int8, na_count=0, n_uniq=32
Column C4: float64 -> int16, na_count=0, n_uniq=1532
Column C5: float64 -> int16, na_count=0, n_uniq=353
Column C6: float64 -> int16, na_count=0, n_uniq=1574
Column C7: float64 -> int16, na_count=0, n_uniq=1335
Column C8: float64 -> int16, na_count=0, n_uniq=1419
Column C9: float64 -> int16, na_count=0, n_uniq=360
Column D1: float64 -> float16, na_count=7300, n_uniq=643
Column D10: float64 -> float16, na_count=88567, n_uniq=1076
Column D11: float64 -> float16, na_count=455805, n_uniq=888
Column D12: float64 -> float16, na_count=963260, n_uniq=869
Column D13: float64 -

Column V184: float64 -> float16, na_count=821037, n_uniq=18
Column V185: float64 -> float16, na_count=821037, n_uniq=33
Column V186: float64 -> float16, na_count=820866, n_uniq=41
Column V187: float64 -> float16, na_count=820866, n_uniq=216
Column V188: float64 -> float16, na_count=821037, n_uniq=46
Column V189: float64 -> float16, na_count=821037, n_uniq=46
Column V19: float64 -> float16, na_count=88662, n_uniq=14
Column V190: float64 -> float16, na_count=820866, n_uniq=226
Column V191: float64 -> float16, na_count=820866, n_uniq=23
Column V192: float64 -> float16, na_count=820866, n_uniq=46
Column V193: float64 -> float16, na_count=820866, n_uniq=39
Column V194: float64 -> float16, na_count=821037, n_uniq=9
Column V195: float64 -> float16, na_count=821037, n_uniq=20
Column V196: float64 -> float16, na_count=820866, n_uniq=43
Column V197: float64 -> float16, na_count=821037, n_uniq=16
Column V198: float64 -> float16, na_count=821037, n_uniq=30
Column V199: float64 -> float16, na_count

Column V307: float64 -> float32, na_count=15, n_uniq=54582
Column V308: float64 -> float32, na_count=15, n_uniq=35249
Column V309: float64 -> float32, na_count=15, n_uniq=6164
Column V31: float64 -> float16, na_count=88662, n_uniq=14
Column V310: float64 -> float32, na_count=15, n_uniq=26505
Column V311: float64 -> float32, na_count=15, n_uniq=4609
Column V312: float64 -> float32, na_count=15, n_uniq=11367
Column V313: float64 -> float32, na_count=7300, n_uniq=8055
Column V314: float64 -> float32, na_count=7300, n_uniq=16327
Column V315: float64 -> float32, na_count=7300, n_uniq=10174
Column V316: float64 -> float32, na_count=15, n_uniq=14985
Column V317: float64 -> float32, na_count=15, n_uniq=22593
Column V318: float64 -> float32, na_count=15, n_uniq=18608
Column V319: float64 -> float32, na_count=15, n_uniq=8173
Column V32: float64 -> float16, na_count=88662, n_uniq=24
Column V320: float64 -> float32, na_count=15, n_uniq=10834
Column V321: float64 -> float32, na_count=15, n_uniq=938

Column id_22: float64 -> float16, na_count=1087000, n_uniq=36
Column id_23: object -> object, na_count=1087000, n_uniq=4
Column id_24: float64 -> float16, na_count=1087744, n_uniq=18
Column id_25: float64 -> float16, na_count=1087060, n_uniq=441
Column id_26: float64 -> float16, na_count=1087021, n_uniq=116
Column id_27: object -> object, na_count=1087000, n_uniq=3
Column id_28: object -> object, na_count=819475, n_uniq=3
Column id_29: object -> object, na_count=819475, n_uniq=3
Column id_30: object -> object, na_count=949007, n_uniq=88
Column id_31: object -> object, na_count=820324, n_uniq=173
Column id_32: float64 -> float16, na_count=948974, n_uniq=7
Column id_33: object -> object, na_count=953271, n_uniq=462
Column id_34: object -> object, na_count=947251, n_uniq=5
Column id_35: object -> object, na_count=819269, n_uniq=3
Column id_36: object -> object, na_count=819269, n_uniq=3
Column id_37: object -> object, na_count=819269, n_uniq=3
Column id_38: object -> object, na_count=8192

In [16]:
train = train_test[:590540]
test = train_test[590540:].drop('isFraud', axis=1)

del train_test
gc.collect()

train.shape, test.shape

((590540, 434), (506691, 433))

## Filling Cards Nans

So the whole idea behind the function count_unqies is to find out for each value card1 how many values does card2,card3,card4, card5 and card5 have in common. For example run the folowing the code seperately and 

unique_train = []
unique_test = []
for value in train['card1'].unique():
    unique_train.append(train['card2'][train['card1'] == value].value_counts().shape[0])
    
for value in test['card1'].unique():
    unique_test.append(test['card2'][test['card1'] == value].value_counts().shape[0])
    
pair_values_train = pd.Series(data=unique_train, index=train['card1'].unique())
pair_values_test = pd.Series(data=unique_test, index=test['card1'].unique())
pd.concat([pair_values_train.value_counts(), pair_values_test.value_counts()], axis=1).rename(columns={0: 'train', 1: 'test'})

pair_values_train[pair_values_train.values == 2]
train['card2'][train['card1'] == 2581].value_counts()

We could see the two distinct values for the value of card1 = 2581 in card2.

https://www.kaggle.com/grazder/filling-card-nans



In [63]:
def count_uniques(train, test, pair):
    unique_train = []
    unique_test = []

    for value in train[pair[0]].unique():
        unique_train.append(train[pair[1]][train[pair[0]] == value].value_counts().shape[0])

    for value in test[pair[0]].unique():
        unique_test.append(test[pair[1]][test[pair[0]] == value].value_counts().shape[0])

    pair_values_train = pd.Series(data=unique_train, index=train[pair[0]].unique())
    pair_values_test = pd.Series(data=unique_test, index=test[pair[0]].unique())
    
    return pair_values_train, pair_values_test

The fill_card_nans get the pair_values_train and pair_values_test from the count_uniques and then it fills all the values of card2,card3..card6 with each value that corresponds in card1. Note card1 values that have only a single occurence are taken here I guess. Becuase the for loop is taking into account values that have pair_values_train == 1. I am not sure about the occurences for if a card1 value has say 2-3 distinct values for card2,card3...card6

However we can see below that in the first part of the function we can see the amount isNan's in training and testing data in the range of 8933 and 8654. However it has been bought down to 4780, 5511, this corresponds to the idea of those unique values in the case of 2 occurences and 3 occurences

In [102]:
def fill_card_nans(train, test, pair_values_train, pair_values_test, pair):
    print(f'In train{[pair[1]]} there are {train[pair[1]].isna().sum()} NaNs' )
    print(f'In test{[pair[1]]} there are {test[pair[1]].isna().sum()} NaNs' )

    print('Filling train...')
    
    for value in pair_values_train[pair_values_train == 1].index:
        train[pair[1]][train[pair[0]] == value] = train[pair[1]][train[pair[0]] == value].value_counts().index[0]
        
    print('Filling test...')

    for value in pair_values_test[pair_values_test == 1].index:
        test[pair[1]][test[pair[0]] == value] = test[pair[1]][test[pair[0]] == value].value_counts().index[0]
        
    print(f'In train{[pair[1]]} there are {train[pair[1]].isna().sum()} NaNs' )
    print(f'In test{[pair[1]]} there are {test[pair[1]].isna().sum()} NaNs' )
    
    return train, test


The below function nans_distribtution gives the number of nan's values for each card2,card3...card6 returns the distrbution 
of the number of Nan's in 0,1 & 2 categoires.

In [103]:
def nans_distribution(train, test, unique_train, unique_test, pair):
    train_nans_per_category = []
    test_nans_per_category = []

    for value in unique_train.unique():
        train_nans_per_category.append(train[train[pair[0]].isin(list(unique_train[unique_train == value].index))][pair[1]].isna().sum())

    for value in unique_test.unique():
        test_nans_per_category.append(test[test[pair[0]].isin(list(unique_test[unique_test == value].index))][pair[1]].isna().sum())

    pair_values_train = pd.Series(data=train_nans_per_category, index=unique_train.unique())
    pair_values_test = pd.Series(data=test_nans_per_category, index=unique_test.unique())
    
    return pair_values_train, pair_values_test

In [104]:
%%time
for card in ['card2','card3','card4','card5','card6']:
    unique_values_train, unique_values_test = count_uniques(train, test, ('card1', card))
    train, test = fill_card_nans(train, test, unique_values_train, unique_values_test, ('card1', card))

In train['card2'] there are 8933 NaNs
In test['card2'] there are 8654 NaNs
Filling train...
Filling test...
In train['card2'] there are 4780 NaNs
In test['card2'] there are 5511 NaNs
In train['card3'] there are 1565 NaNs
In test['card3'] there are 3002 NaNs
Filling train...
Filling test...
In train['card3'] there are 17 NaNs
In test['card3'] there are 48 NaNs
In train['card4'] there are 1577 NaNs
In test['card4'] there are 3086 NaNs
Filling train...
Filling test...
In train['card4'] there are 27 NaNs
In test['card4'] there are 130 NaNs
In train['card5'] there are 4259 NaNs
In test['card5'] there are 4547 NaNs
Filling train...
Filling test...
In train['card5'] there are 939 NaNs
In test['card5'] there are 1449 NaNs
In train['card6'] there are 1571 NaNs
In test['card6'] there are 3007 NaNs
Filling train...
Filling test...
In train['card6'] there are 26 NaNs
In test['card6'] there are 54 NaNs
Wall time: 53min 8s


## Finding UID

In [105]:
train[cols].head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14
0,2987000,0,86400,68.5,W,13926,327.0,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,1,1,0,0,0,1,0,0,1,0,2,0,1,1
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,1,1,0,0,0,1,0,0,0,0,1,0,1,1
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,315.0,NaN,NaN,NaN,315.0,1,1,0,0,0,1,0,0,1,0,1,0,1,1
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,NaN,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,84.0,NaN,NaN,NaN,NaN,111.0,2,5,0,0,0,4,0,0,1,0,1,0,25,1
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,NaN,gmail.com,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,0,0,1,0,1,0,1,1,0,1,1


D1 column holds the cumulative number of days since the day the transanction started. Therefore subtracting that from the starting day gives the number of days since the card has been operational

In [112]:
for df in [train,test]:
    df['DaysFromStart'] = np.floor(df['TransactionDT']/(60*60*24)) - 1
    df['D1-DaysFromStart'] = df['D1'] - df['DaysFromStart']
cols = cols + ['DaysFromStart','D1-DaysFromStart']
train[cols].head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,DaysFromStart,D1-DaysFromStart
0,2987000,0,86400,68.5,W,13926,327.0,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,1,1,0,0,0,1,0,0,1,0,2,0,1,1,0.0,14.0
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,1,1,0,0,0,1,0,0,0,0,1,0,1,1,0.0,0.0
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,315.0,NaN,NaN,NaN,315.0,1,1,0,0,0,1,0,0,1,0,1,0,1,1,0.0,0.0
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,NaN,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,84.0,NaN,NaN,NaN,NaN,111.0,2,5,0,0,0,4,0,0,1,0,1,0,25,1,0.0,112.0
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,NaN,gmail.com,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,0,0,1,0,1,0,1,1,0,1,1,0.0,0.0


Next finding the Unique Idnetification is the distinguishing step that made the difference in the competition. It was found  that certain columns clubbed together could help indentify unique transaction ID's. 

In [113]:
for df in [train,test]:
    df['uid'] = df['ProductCD'].astype(str) + '_' + df['card1'].astype(str) + '_' + df['card2'].astype(str)
    df['uid'] = df['uid'] + '_' + df['card3'].astype(str) + '_' + df['card4'].astype(str)
    df['uid'] = df['uid'] + '_' + df['card5'].astype(str) + '_' + df['card6'].astype(str)
    df['uid'] = df['uid'] + '_' + df['addr1'].astype(str) + '_' + df['D1-DaysFromStart'].astype(str)
cols = ['uid'] + cols
train[cols].head()

,uid,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,DaysFromStart,D1-DaysFromStart
0,W_13926_327.0_150.0_discover_142.0_credit_315.0_14.0,2987000,0,86400,68.5,W,13926,327.0,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,1,1,0,0,0,1,0,0,1,0,2,0,1,1,0.0,14.0
1,W_2755_404.0_150.0_mastercard_102.0_credit_325.0_0.0,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,1,1,0,0,0,1,0,0,0,0,1,0,1,1,0.0,0.0
2,W_4663_490.0_150.0_visa_166.0_debit_330.0_0.0,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,315.0,NaN,NaN,NaN,315.0,1,1,0,0,0,1,0,0,1,0,1,0,1,1,0.0,0.0
3,W_18132_567.0_150.0_mastercard_117.0_debit_476.0_112.0,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,NaN,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,84.0,NaN,NaN,NaN,NaN,111.0,2,5,0,0,0,4,0,0,1,0,1,0,25,1,0.0,112.0
4,H_4497_514.0_150.0_mastercard_102.0_credit_420.0_0.0,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,NaN,gmail.com,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,0,0,1,0,1,0,1,1,0,1,1,0.0,0.0


In [114]:
by = ['uid']
grouped = train.groupby(by, as_index=False)['isFraud'].agg(['count','sum']).reset_index()
grouped.sort_values(by).head(10)

,uid,count,sum
0,C_10003_555.0_128.0_visa_226.0_debit_nan_-89.0,5,0
1,C_1000_555.0_185.0_mastercard_224.0_debit_nan_-65.0,1,0
2,C_10022_555.0_117.0_mastercard_224.0_debit_nan_-34.0,1,0
3,C_10023_111.0_150.0_visa_226.0_debit_nan_-102.0,1,0
4,C_10023_111.0_150.0_visa_226.0_debit_nan_-114.0,1,0
5,C_10023_111.0_150.0_visa_226.0_debit_nan_-145.0,2,0
6,C_10023_111.0_150.0_visa_226.0_debit_nan_-176.0,1,0
7,C_10023_111.0_150.0_visa_226.0_debit_nan_-72.0,1,0
8,C_10024_321.0_150.0_visa_144.0_credit_nan_-136.0,3,3
9,C_10024_321.0_150.0_visa_144.0_credit_nan_-160.0,1,0


In the above table we can see that the UID number 8 has 3 counts and 3 sum. The sum of 3 shows that all the transactions from that account has been fradulent. This is a pretty amazing find. I am pretty sure this is a good way to indentify rows that have been fraudulent. The whole idea behind the competition is to find fradulent credit cards

In [115]:
grouped.describe()

,count,sum
count,231339.000000,231339.000000
mean,2.552704,0.089319
std,5.207512,0.906427
min,1.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,0.000000
75%,2.000000,0.000000
max,1414.000000,90.000000


In [116]:
train = train.drop(['DaysFromStart','D1-DaysFromStart'], axis=1)
test = test.drop(['DaysFromStart','D1-DaysFromStart'], axis=1)

In [118]:
import pickle
train.to_pickle('I:/ML Datasets/IEEE Fradulent/train_reduced.pkl')
test.to_pickle('I:/ML Datasets/IEEE Fradulent/test_reduced.pkl')